## NAMES CLASSIFICATION:

### IMPORT MODULES:

In [1]:
import csv
import collections

import math as mh
import numpy as np
import string as st
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from time import time
from itertools import chain
from itertools import islice

### DATA LISTS:

In [2]:
ingridients = ["агар", "соль", "приправа", "специи", "кориц", "сода", "лаванда"]

dishes = ["овощи", "хычан", "булочка", "салат"]

noneating = ["мыло"]

groceries = ["чай", "кофе", "вода", "чипс", "конфет", "икра", "печень", "горбуша", "пиво", "спаржа", "горошек", "ягнен",
             "хлеб", "лед", "сэндвич", "яйцо", "торт", "напиток", "лапша", "суп", "филе", "коктейль", "шашлык", "борщ",
             "плов", "паприкаш", "рис", "болоньезе", "сыр", "пита", "колбас", "дайфук", "стейк", "кефир", "молоко",
             "сметана", "ряженка", "творог", "шницель", "биточ", "какао", "цыплен", "фрикадельки", "драники",
             "мороженое", "пломбир", "снежок", "простокваша", "варенец", "шоколад"]

fruits = ["абрикос", "авокад", "алыч", "ананас", "аннон", "апельсин", "банан", "бергамот", "гранат", "грейпфрут", "груш",
          "гуав", "джекфрут", "гуанабан", "питаха", "дуриан", "карамбол", "киви", "клементин", "кумкват", "лайм", "лимон",
          "лич", "лонган", "манго", "мангостин", "мандарин", "маракуй", "мушмул", "нектарин", "папай", "персик", "помел"
          "померан", "рамбутан", "салак", "саподилл", "свит", "слив", "танжел", "хеномелес", "хурм", "цитрон", "черимой",
          "чомп", "ябло", "гранадилл", "помел"]

vegatables = ["аррорут", "баклажан", "бами", "батат", "брюкв", "дайкон", "кабач", "каперс", "капуст", "броккол",
              "картоф", "кукуруз", "лук", "люфф", "морков", "момордик", "огур", "пастерн", "патисс", "перец","помидор",
              "радикк", "редис", "редьк", "реп", "свек", "топинам", "горох", "нут", "тыкв", "фенхе", "хрен", "цукки",
              "чайот", "чесн", "ямс", "гриб", "томат", "кинза", "петрушка", "укроп", "шпинат", "имбирь"]

berries = ["арбуз", "барбарис", "боярышник", "брусник", "бузин", "виноград", "годж", "голуби", "вишн", "дын", "ежеви",
           "земляни", "ирг", "калин", "кизил", "клубни", "клюкв", "смороди", "крыжовни", "лимонни", "малин", "морош",
           "облепих", "оливк", "ряби", "терн", "фейхо", "физалис", "черешн", "черни", "черему", "шелковиц", "шиповни",
           "жимолость"]

### READ CSVS:

In [3]:
with open("../data/vkusvill_items.csv", 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    catalog_df = pd.DataFrame(reader)

catalog_df = catalog_df.rename(columns = {"": "shop_id", "item_composition_txt": "item_composition", "nutrion_value_txt" : "nutrion_value"})

In [4]:
categories_df = pd.read_csv("../data/vkusvill_categories.csv")
categories_df = categories_df.rename( columns = {"Unnamed: 0": "shop_id"})

### PAIRS ITEM NAMES:

In [5]:
def clear_item_name(item_name):
    answer = ""
    
    for symbol in item_name:
        if symbol == '_':
            answer += ' '
            continue  
        elif symbol in [
            '\\', 
            '(', 
            ')', 
            '.', 
            ',', 
            'г'
        ] or symbol.isdigit():
            answer += ' '
            continue
        else:
            answer += symbol

    return answer

In [6]:
def find_number_in_string(string):
    answer = ""
    index = 0
    
    for i in string:
        if (i >= '0' and i <= '9') or i == ',':
            answer += i

    while index < len(answer) and (answer[index] == ' ' or answer[index] == ','):
        answer = answer[:index] + answer[index + 1:]

    return ("0" if answer == "" else answer.lower())

In [7]:
def find_tokens(catalog_df):
    tokens, have_weight, have_count, have_percents = pd.Series(), pd.Series(), pd.Series(), pd.Series()
    
    for i in range(len(catalog_df.index)):
        tokens = pd.concat([tokens, pd.Series(clear_item_name(catalog_df["item_name"][i]).lower())], ignore_index=True)

        if '%' in catalog_df["item_name"][i]:
            have_percents = pd.concat([have_percents, pd.Series(find_number_in_string(catalog_df["item_name"][i]))], ignore_index=True)
        elif ' шт' in catalog_df["item_name"][i]:
             have_count = pd.concat([have_count, pd.Series(find_number_in_string(catalog_df["item_name"][i]))], ignore_index=True)
        elif ' г' in catalog_df["item_name"][i]:
            have_weight = pd.concat([have_weight, pd.Series(find_number_in_string(catalog_df["item_name"][i]))], ignore_index=True)
        else:
            have_weight = pd.concat([have_weight, pd.Series("0")], ignore_index=True)
            have_count = pd.concat([have_count, pd.Series("0")], ignore_index=True)
            have_percents = pd.concat([have_percents, pd.Series("0")], ignore_index=True)
    
    return tokens, have_weight, have_count, have_percents

tokens, have_weight, have_count, have_percents = find_tokens(catalog_df)
prepare_catalog_df = pd.concat([catalog_df.item_name, tokens, have_weight, have_count, have_percents], axis=1, ignore_index=True)
prepare_catalog_df = prepare_catalog_df.rename(columns = {0: "item_name", 1: "clean_name", 2: "г", 3: "шт", 4: "%"})

In [8]:
prepare_catalog_df.head(25)

,item_name,clean_name,г,шт,%
0,Конфета жевательная с ароматом клубники,конфета жевательная с ароматом клубники,0,0,0
1,Смесь овощная \Три капусты\(Продукт замороженный),смесь овощная три капусты продукт замороженный,0,0,0
2,"Чипсы из морской капусты с анчоусами, 20 г",чипсы из морской капусты с анчоусами,20,0,0
3,Салат свекольный с грецким орехом,салат свекольный с рецким орехом,0,0,0
4,"Кета малосоленая филе-ломтики, 150 г",кета малосоленая филе-ломтики,150,0,0
5,"Тунец холодного копчения карпаччо с кунжутом, ...",тунец холодно о копчения карпаччо с кунжутом ...,100,0,0
6,Филе окорочка цыпленка-бройлера с овощами,филе окорочка цыпленка-бройлера с овощами,0,0,0
7,Нут жареный,нут жареный,0,0,0
8,Хлебцы ржаные с паприкой,хлебцы ржаные с паприкой,0,0,0
9,Пирожное \Ежик\,пирожное ежик,0,0,5


In [9]:
def it_includes(item_name, lst):
    for i in lst:
        if i in item_name.lower():
            return True

    return False

In [10]:
name_count = 0
nutrion_count = 0

def is_eating(catalog_df, categories_df, item_id):

    global name_count
    global nutrion_count
    
    ind = catalog_df[catalog_df["item_id"] == str(item_id)]["nutrion_value"].index[0]

    if it_includes(catalog_df["item_name"][ind].lower(), noneating):
        name_count += 1
        return False
    elif it_includes(catalog_df["item_name"][ind].lower(), fruits):
        name_count += 1
        return True
    elif it_includes(catalog_df["item_name"][ind].lower(), vegatables):
        name_count += 1
        return True
    elif it_includes(catalog_df["item_name"][ind].lower(), berries):
        name_count += 1
        return True
    elif it_includes(catalog_df["item_name"][ind].lower(), ingridients):
        name_count += 1
        return True
    elif it_includes(catalog_df["item_name"][ind].lower(), dishes):
        name_count += 1
        return True
    elif it_includes(catalog_df["item_name"][ind].lower(), groceries):
        name_count += 1
        return True

    nutrion_count += 1
    return not catalog_df[catalog_df["item_id"] == str(item_id)]["nutrion_value"][ind] == ""

In [11]:
target = pd.Series()

def binary_classification_products(catalog_df, categories_df, target):

    for i in range(len(catalog_df.index)):
        target = pd.concat([target, pd.Series(is_eating(catalog_df, categories_df, catalog_df.iloc[i]["item_id"]))], ignore_index=True)

    catalog_df = pd.concat([catalog_df.item_name, target], axis=1, ignore_index=True)
    
    return catalog_df

classification = binary_classification_products(catalog_df, categories_df, target)

classification_with_data = pd.concat([catalog_df, classification], axis = 1)
classification_with_data = classification_with_data.rename(columns = {1 : "eating"})
classification_with_data = classification_with_data[["item_id", "item_name", "item_composition", "eating"]]

In [12]:
def calculate_result(classification_with_data, name_count, nutrion_count):
    all = classification_with_data.shape[0]
    eating = classification_with_data[classification_with_data["eating"] == True].shape[0]
    noneating = classification_with_data[classification_with_data["eating"] == False].shape[0]
    print(f"All items: {all}")
    print(f"Eating products: {round(eating / all, 3) * 100} %")
    print(f"Noneating products: {round(noneating / all, 3) * 100} %")
    print(f"Classified products by name: {round(name_count / all, 3) * 100} %")
    print(f"Classified products by nutrion: {round(nutrion_count / all, 3) * 100} %")

calculate_result(classification_with_data, name_count, nutrion_count)

All items: 11418
Eating products: 70.89999999999999 %
Noneating products: 29.099999999999998 %
Classified products by name: 53.2 %
Classified products by nutrion: 46.800000000000004 %


In [13]:
step = 55
count = 0

classification_with_data[classification_with_data["eating"] == False][step * count:step * (count + 1)]

,item_id,item_name,item_composition,eating
10,39018,"L-карнитин 3600 мг в 1 ампуле, 25 мл","вода подготовленная, L-карнитин, регулятор кис...",False
59,39564,"Салфетки влажные антисептические спирт., 50 шт","нетканое полотно, лосьон пропитывающий (спирт ...",False
65,39671,Витамин С 900 мг таблетки шипучие,"L-аскорбиновая кислота, лимонная кислота (регу...",False
88,40211,Маска гидрогелевая для лица с золотом,"вода,глицерин, дипропиленгликоль,бутилен глико...",False
93,40277,"Бумага туалетная 4 слоя белая, 4 рулона",100% первичная целлюлоза,False
120,41237,Подгузники Merries L 9-14 кг 64 шт,"нетканый полиолефин, полиолефин-нетканый полиэ...",False
126,41587,"Мульти комплекс витамины и минералы, 60 капсул","магния цитрат; растительная капсула, получаема...",False
141,42371,"Глюкозамин, хондроитин МСМ, гиалуроновая кисло...","глюкозамина сульфат, хондроитина сульфат, мети...",False
150,42673,"Набор алюминиевых круглых форм, D20,5 см, 1405...",листовой алюминий,False
209,43428,"Блокнот в клетку, 40 л.",бумага,False


### EMPTY TARGETS:

ids = [76793, 76819, 76842]